In [0]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

In [0]:
from google.colab import files
upload = files.upload()

Saving hw_sw lab_testdata_final.csv to hw_sw lab_testdata_final.csv


In [100]:
#load data from file with data
column_names = ["g_x", "g_y", "g_z", "a_x", "a_y", "a_z","target"]
df = pd.read_csv("hw_sw lab_testdata_final.csv", names = column_names)
target = df.target
print(df.head(40))

      g_x    g_y    g_z    a_x    a_y    a_z  target
0   -1093    206   -316  13936   6432  -7140       1
1   -1165    351    -39  13660   6556  -7112       1
2   -1533    117     40  13196   6740  -6600       1
3   -2799  -1531   2984  19288  -1284  -5188       1
4    8267   -727 -10765  17252 -21412 -17264       1
5    9508   9230 -13284   6756  12048 -13620       1
6     708   4749  -2169  10924  13404  -8192       1
7    3479   6464  -1774  11304  13100  -6580       1
8     563    582   3781  13372   7468  -5148       1
9   -1452    330   -396  15380   7604  -4680       1
10   -739    147   -144  11652   7640  -8576       0
11   -696   -149   -553  12076   7868  -9512       0
12  -1146    427   -180  11968   8008  -9004       0
13  -1280    316     32  11752   7604  -8220       0
14   -840      5   -934  11664   8184  -9312       0
15  -1005    133   -205  12120   8064  -9500       0
16   -926    -73   -269  11852   7856  -9492       0
17   -938    268   -136  12024   7888  -8964  

In [101]:
#time steps is based on how many time points you want to put into one entry 
N_TIME_STEPS = 10
#number of features, should be 6 since we have acceleration and gyroscope for xyz 
N_FEATURES = 6
step = 1
segments = []
labels = []
for i in range(0, len(df) - N_TIME_STEPS, step):
    a_x = df['a_x'].values[i: i + N_TIME_STEPS]
    a_y = df['a_y'].values[i: i + N_TIME_STEPS]
    a_z = df['a_z'].values[i: i + N_TIME_STEPS]
    g_x = df['g_x'].values[i: i + N_TIME_STEPS]
    g_y = df['g_y'].values[i: i + N_TIME_STEPS]
    g_z = df['g_z'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['target'][i: i + N_TIME_STEPS])[0][0]
    segments.append([g_x, g_y, g_z,a_x, a_y, a_z,])
    labels.append(label)

print(np.array(segments).shape)

#get the desired shape for the neural network
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
#labels width is however many gestures we have
labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

(1530, 6, 10)


In [102]:
print(reshaped_segments[0])
print(labels[0])

[[ -1093.  -1165.  -1533.  -2799.   8267.   9508.]
 [   708.   3479.    563.  -1452.    206.    351.]
 [   117.  -1531.   -727.   9230.   4749.   6464.]
 [   582.    330.   -316.    -39.     40.   2984.]
 [-10765. -13284.  -2169.  -1774.   3781.   -396.]
 [ 13936.  13660.  13196.  19288.  17252.   6756.]
 [ 10924.  11304.  13372.  15380.   6432.   6556.]
 [  6740.  -1284. -21412.  12048.  13404.  13100.]
 [  7468.   7604.  -7140.  -7112.  -6600.  -5188.]
 [-17264. -13620.  -8192.  -6580.  -5148.  -4680.]]
[0. 1. 0.]


In [103]:
X_train, X_test, y_train, y_test = train_test_split(
        reshaped_segments, labels, test_size=0.15, random_state=20)
print(X_train.shape)
print(X_test.shape)

(1300, 10, 6)
(230, 10, 6)


In [0]:
N_CLASSES = 3
N_HIDDEN_UNITS = 64
def create_LSTM_model(inputs):
    W = {
        'hidden': tf.Variable(tf.random_normal([N_FEATURES, N_HIDDEN_UNITS])),
        'output': tf.Variable(tf.random_normal([N_HIDDEN_UNITS, N_CLASSES]))
    }
    biases = {
        'hidden': tf.Variable(tf.random_normal([N_HIDDEN_UNITS], mean=1.0)),
        'output': tf.Variable(tf.random_normal([N_CLASSES]))
    }
    
    X = tf.transpose(inputs, [1, 0, 2])
#     X = tf.transpose(inputs, [1, 0, 3])
    X = tf.reshape(X, [-1, N_FEATURES])
    hidden = tf.nn.relu(tf.matmul(X, W['hidden']) + biases['hidden'])
    hidden = tf.split(hidden, N_TIME_STEPS, 0)

    # Stack 2 LSTM layers
    lstm_layers = [tf.contrib.rnn.BasicLSTMCell(N_HIDDEN_UNITS, forget_bias=1.0) for _ in range(2)]
    lstm_layers = tf.contrib.rnn.MultiRNNCell(lstm_layers)

    outputs, _ = tf.contrib.rnn.static_rnn(lstm_layers, hidden, dtype=tf.float32)

    # Get output for the last time step
    lstm_last_output = outputs[-1]

    return tf.matmul(lstm_last_output, W['output']) + biases['output']

In [0]:
print(X)

Tensor("input:0", shape=(?, 10, 6), dtype=float32)


In [0]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, N_TIME_STEPS, N_FEATURES], name="input")
Y = tf.placeholder(tf.float32, [None, N_CLASSES])

In [0]:
print(X)
print("Y")
print(Y)

Tensor("input:0", shape=(?, 10, 6), dtype=float32)
Y
Tensor("Placeholder:0", shape=(?, 2), dtype=float32)


In [0]:
pred_Y = create_LSTM_model(X)

pred_softmax = tf.nn.softmax(pred_Y, name="y_")

In [107]:
print(pred_Y)
print(pred_softmax)

Tensor("add_1:0", shape=(?, 3), dtype=float32)
Tensor("y_:0", shape=(?, 3), dtype=float32)


In [0]:
L2_LOSS = 0.0015

l2 = L2_LOSS * \
    sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred_Y, labels = Y)) + l2

In [0]:
#Change learning rate if you want
# LEARNING_RATE = 0.0025
LEARNING_RATE = 0.005
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred_softmax, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

In [111]:
#Change epochs to as much or few as you want
#Batch size whatever you want
N_EPOCHS = 400
BATCH_SIZE =64
saver = tf.train.Saver()

history = dict(train_loss=[], 
                     train_acc=[], 
                     test_loss=[], 
                     test_acc=[])

# sess=tf.InteractiveSession()
sess=tf.Session()
sess.run(tf.global_variables_initializer())

train_count = len(X_train)

for i in range(1, N_EPOCHS + 1):
    for start, end in zip(range(0, train_count, BATCH_SIZE),
                          range(BATCH_SIZE, train_count + 1,BATCH_SIZE)):
        sess.run(optimizer, feed_dict={X: X_train[start:end],
                                       Y: y_train[start:end]})

    _, acc_train, loss_train = sess.run([pred_softmax, accuracy, loss], feed_dict={
                                            X: X_train, Y: y_train})

    _, acc_test, loss_test = sess.run([pred_softmax, accuracy, loss], feed_dict={
                                            X: X_test, Y: y_test})

    history['train_loss'].append(loss_train)
    history['train_acc'].append(acc_train)
    history['test_loss'].append(loss_test)
    history['test_acc'].append(acc_test)

    if i != 1 and i % 10 != 0:
        continue

    print(f'epoch: {i} test accuracy: {acc_test} loss: {loss_test}')
    #saves the final version of the model
    #which creates 4 files!!!
    if(i==N_EPOCHS):
      saver.save(sess, 'LSTM_v2', global_step=i)
predictions, acc_final, loss_final = sess.run([pred_softmax, accuracy, loss], feed_dict={X: X_test, Y: y_test})

print()
print(f'final results: accuracy: {acc_final} loss: {loss_final}')


print()

epoch: 1 test accuracy: 0.7434782385826111 loss: 1.3070101737976074
epoch: 10 test accuracy: 0.708695650100708 loss: 1.104088544845581
epoch: 20 test accuracy: 0.804347813129425 loss: 0.9291523694992065
epoch: 30 test accuracy: 0.8608695864677429 loss: 0.8510252237319946
epoch: 40 test accuracy: 0.8304347991943359 loss: 0.9740810394287109
epoch: 50 test accuracy: 0.8391304612159729 loss: 0.941743016242981
epoch: 60 test accuracy: 0.8260869383811951 loss: 0.9465214014053345
epoch: 70 test accuracy: 0.821739137172699 loss: 0.9955428242683411
epoch: 80 test accuracy: 0.8173912763595581 loss: 0.9092360734939575
epoch: 90 test accuracy: 0.856521725654602 loss: 0.8451579809188843
epoch: 100 test accuracy: 0.8782608509063721 loss: 0.7906525135040283
epoch: 110 test accuracy: 0.873913049697876 loss: 0.8868926763534546
epoch: 120 test accuracy: 0.873913049697876 loss: 0.844433605670929
epoch: 130 test accuracy: 0.8608695864677429 loss: 0.871934175491333
epoch: 140 test accuracy: 0.8652173876762

In [92]:
print(sess)
print(predictions.shape)
print(y_test.shape)

(230, 3)
(230, 3)


In [93]:
print(predictions[200:230])
print(y_test[200:230])

[[9.50715959e-01 4.80376743e-02 1.24637235e-03]
 [6.20382011e-01 1.06834315e-01 2.72783637e-01]
 [8.03280592e-01 1.94324329e-01 2.39502382e-03]
 [3.60560685e-01 5.98982751e-01 4.04565223e-02]
 [3.38516489e-04 9.87386763e-01 1.22746630e-02]
 [9.17720318e-01 4.00809608e-02 4.21987586e-02]
 [9.99772131e-01 2.22524832e-04 5.34491892e-06]
 [9.64710772e-01 7.91223068e-03 2.73769125e-02]
 [1.36011928e-01 8.63898098e-01 8.98873986e-05]
 [9.97836292e-01 2.09985184e-03 6.38911151e-05]
 [9.64940071e-01 4.76994365e-03 3.02899573e-02]
 [9.98179078e-01 1.81207946e-03 8.81748383e-06]
 [9.99932051e-01 4.06106956e-05 2.73387832e-05]
 [8.57815333e-03 3.56944976e-04 9.91064906e-01]
 [7.65268385e-01 3.07824723e-02 2.03949153e-01]
 [9.99461710e-01 4.74777276e-04 6.35198303e-05]
 [5.72346985e-01 4.94860895e-02 3.78166884e-01]
 [7.01653166e-03 5.39544582e-01 4.53438908e-01]
 [4.71625537e-01 7.96000299e-04 5.27578413e-01]
 [9.99256313e-01 7.18457275e-04 2.53019498e-05]
 [6.36459095e-04 9.98344898e-01 1.018613

In [0]:
sess.close()

In [115]:
#this is reloading the model in and running it.
with tf.Session() as sess2:
  new_saver = tf.train.import_meta_graph('LSTM_v2-400.meta')
  new_saver.restore(sess2, tf.train.latest_checkpoint('./'))
  predictions1, acc_final1, loss_final1 = sess2.run([pred_softmax, accuracy, loss], feed_dict={X: X_test, Y: y_test})
  predictions2 = sess2.run(X_test)

INFO:tensorflow:Restoring parameters from ./LSTM_v2-400


In [118]:
print(f'final results: accuracy: {acc_final1} loss: {loss_final1}')
print(predictions)

final results: accuracy: 0.904347836971283 loss: 0.7533988952636719
[[9.90304410e-01 9.64471046e-03 5.09339297e-05]
 [9.99753535e-01 1.14026458e-04 1.32412722e-04]
 [4.73897606e-02 2.58629739e-01 6.93980515e-01]
 [9.97711182e-01 2.17203586e-03 1.16736832e-04]
 [9.98575926e-01 1.86912017e-04 1.23719755e-03]
 [9.56698477e-01 4.84480086e-04 4.28169817e-02]
 [9.98867154e-01 1.49726649e-04 9.83143807e-04]
 [1.70926820e-03 7.32694520e-04 9.97557998e-01]
 [9.86371100e-01 1.22589087e-02 1.37000019e-03]
 [2.15044175e-03 9.71044958e-01 2.68046483e-02]
 [9.99420643e-01 2.45088508e-04 3.34331504e-04]
 [9.97616053e-01 2.13965192e-04 2.17000907e-03]
 [1.44476420e-03 8.00405264e-01 1.98150009e-01]
 [6.35574199e-03 9.93446171e-01 1.98052148e-04]
 [9.93826449e-01 6.15865644e-03 1.49406214e-05]
 [9.98549521e-01 1.38137187e-03 6.90310917e-05]
 [1.74109577e-04 9.94403243e-01 5.42253256e-03]
 [8.51374399e-03 1.74116431e-04 9.91312087e-01]
 [1.81512877e-01 3.23859113e-03 8.15248489e-01]
 [9.69580829e-01 3.1

In [117]:
# sess2=tf.Session()
# sess2.run(tf.global_variables_initializer())
# # saver=tf.train.Saver()

RuntimeError: ignored